<a href="https://colab.research.google.com/github/UttkarshM/collaborative_filtering/blob/master/Collaborative_Filtering_Movies_kaggleipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Recommendation systems


Recommendation systems are usually based on two types of systems:
1. content based filter
2. collaborative based filter

content based filter recommends you things based on your previous preferences<br><br>
whereas collaborative based filter recommends you things based on the what similar users liked
(the things u liked in the past are used to get the similarity between u and the user).

## Cosine Similarity


Cosine similarity measures the similarity between two vectors of an inner product space. It is measured by the cosine of the angle between two vectors and determines whether two vectors are pointing in roughly the same direction. It is often used to measure document similarity in text analysis.

links:
<br>movie recommendation systems<br>
https://www.freecodecamp.org/news/how-to-build-a-movie-recommendation-system-based-on-collaborative-filtering/

In [ ]:
!pip install fuzzywuzzy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
var =  'drive/MyDrive/Datasets/DataSets/'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

In [ ]:
user_ratings_df = pd.read_csv(var+'/ratings.csv')
user_ratings_df

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
...,...,...,...,...
26024284,270896,58559,5.0,1257031564
26024285,270896,60069,5.0,1257032032
26024286,270896,63082,4.5,1257031764
26024287,270896,64957,4.5,1257033990


In [ ]:
movie_data = pd.read_csv(var+"/movies_metadata.csv")
movie_metadata = movie_data[["title","genres"]]
# movie_metadata.iloc[0]["genres"]
movie_metadata = movie_metadata[:40000]

<ipython-input-9-146212a87e86>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_data = pd.read_csv(var+"/movies_metadata.csv")


In [ ]:
tester = pd.read_csv(var+"/ratings.csv")
tester

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
...,...,...,...,...
26024284,270896,58559,5.0,1257031564
26024285,270896,60069,5.0,1257032032
26024286,270896,63082,4.5,1257031764
26024287,270896,64957,4.5,1257033990


user_ratings_df.columns
user_ratings_df = pd.concat([user_ratings_df,])

In [ ]:
movie_data = pd.concat([user_ratings_df,movie_metadata],axis=1).fillna(0)
# movie_data = movie_data.iloc[:40000]

In [ ]:
movie_data

,userId,movieId,rating,timestamp,title,genres
0,1,110,1.0,1425941529,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,1,147,4.5,1425942435,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,1,858,5.0,1425941523,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,1,1221,5.0,1425941546,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,1,1246,5.0,1425941556,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...,...,...,...
26024284,270896,58559,5.0,1257031564,0,0
26024285,270896,60069,5.0,1257032032,0,0
26024286,270896,63082,4.5,1257031764,0,0
26024287,270896,64957,4.5,1257033990,0,0


In [ ]:
movie_titles = movie_data[["title"]]
movie_titles.to_numpy()

array([['Toy Story'],
       ['Jumanji'],
       ['Grumpier Old Men'],
       ...,
       [0],
       [0],
       [0]], dtype=object)

In [ ]:
for i in movie_metadata.iloc[0]:
  print(i)

Toy Story
[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]


user_ratings_df.columns,user_ratings_df.head()

movie_data = movie_metadata.merge(user_ratings_df,on="movieId")
movie_data.head()

In [ ]:
user_ratings_df = user_ratings_df.iloc[:40000]

In [ ]:
user_item_matrix = user_ratings_df.pivot(index=["userId"],columns=["movieId"],values='rating').fillna(0)
user_item_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,168174,168252,168366,168434,168612,170355,170705,171763,172547,174585
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
429,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
430,3.0,0.0,3.0,0.0,3.0,5.0,5.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
movie_data = movie_data[:40000]

In [ ]:
movie_data = movie_data.dropna(0)

<ipython-input-72-45f7f092fe56>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  movie_data = movie_data.dropna(0)


In [ ]:
user_item_matrix.shape

(432, 5719)

In [ ]:
cf_knn_model = NearestNeighbors(metric="cosine",algorithm="brute",n_neighbors=10,n_jobs=-1)
cf_knn_model.fit(user_item_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [ ]:
user = pd.DataFrame(np.zeros((1,5719)))
print(user.shape,user)
user_item_matrix = pd.concat([user,user_item_matrix],ignore_index=True)

(1, 5719)    0     1     2     3     4     5     6     7     8     9     ...  5709  \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

   5710  5711  5712  5713  5714  5715  5716  5717  5718  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[1 rows x 5719 columns]


In [ ]:
user_item_matrix=user_item_matrix.fillna(0)

In [ ]:
def movie_recommender_engine(movie_name, matrix, cf_model, n_recs):
    # Fit model on matrix
    # cf_knn_model.fit(matrix)

    # Extract input movie ID
    movie_id = process.extractOne(movie_name, movie_data['title'])
    sl = int(movie_id[2])
    it = movie_data.iloc[sl]['movieId']
    # matrix[movie_id]

    user_item_matrix.iloc[0][it]=2
    # Calculate neighbour distances
    distances, indices = cf_model.kneighbors(np.array(user_item_matrix.iloc[0]).reshape(1,-1), n_neighbors=n_recs)
    movie_rec_ids = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]

    # # List to store recommendations
    cf_recs = []
    for i in movie_rec_ids:
        cf_recs.append({'Title':movie_data['title'][i[0]],'Distance':i[1]})

    # # Select top number of recommendations needed
    df = pd.DataFrame(cf_recs, index = range(1,n_recs))

    return df



In [ ]:
def movie_recommender_engine1(movie_name, matrix, cf_model, n_recs):
    # Fit model on matrix
    cf_knn_model.fit(matrix)

    # Extract input movie ID
    movie_id = process.extractOne(movie_name, movie_data['Jumanji'])[2]
    return movie_id

In [ ]:
n_recs = 10
valv = movie_recommender_engine('Tomb Raider', user_item_matrix, cf_knn_model, n_recs)

In [ ]:
valv

,Title,Distance
1,Species,0.970258
2,Belle de Jour,0.960293
3,Dracula: Dead and Loving It,0.959944
4,Leaving Las Vegas,0.959372
5,Eat Drink Man Woman,0.954481
6,Mrs. Parker and the Vicious Circle,0.953209
7,Bad Boys,0.890952
8,Hackers,0.886598
9,Jumanji,0.869842


In [ ]:
movie_data['title'].shape

(26024289,)

In [ ]:
movie_id = process.extractOne('Tomb Raider', movie_data['title'])

In [ ]:
sl = int(movie_id[2])
it = movie_data.iloc[sl]['movieId']

In [ ]:
it

3699

In [ ]:
user_item_matrix[585].unique()


array([0. , 2. , 2.5, 3. , 3.5, 4. , 1. , 5. ])

In [ ]:
movie_data.iloc[585]

userId                                                      11
movieId                                                  55830
rating                                                     3.0
timestamp                                           1231675698
title                                                   Batman
genres       [{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...
Name: 585, dtype: object

In [ ]:
movie_data[['title','movieId']]

,title,movieId
0,Toy Story,110
1,Jumanji,147
2,Grumpier Old Men,858
3,Waiting to Exhale,1221
4,Father of the Bride Part II,1246
...,...,...
39995,Edge of Doom,1017
39996,Youth Runs Wild,1028
39997,Broadway Thru a Keyhole,1035
39998,Endgame,1036


In [ ]:
valv

In [ ]:
user_item_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,168174,168252,168366,168434,168612,170355,170705,171763,172547,174585
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
429,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
430,3.0,0.0,3.0,0.0,3.0,5.0,5.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
movie_data

,userId,movieId,rating,timestamp,title,genres
0,1,110,1.0,1425941529,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,1,147,4.5,1425942435,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,1,858,5.0,1425941523,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,1,1221,5.0,1425941546,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,1,1246,5.0,1425941556,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...,...,...,...
26024284,270896,58559,5.0,1257031564,0,0
26024285,270896,60069,5.0,1257032032,0,0
26024286,270896,63082,4.5,1257031764,0,0
26024287,270896,64957,4.5,1257033990,0,0


In [ ]:
movie_data.info()

In [ ]:
valv.to_numpy().reshape(-1,2)

In [ ]:
user_item_matrix

,0,1,2,3,4,5,6,7,8,9,...,168174,168252,168366,168434,168612,170355,170705,171763,172547,174585
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
429,0.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
430,0.0,3.0,0.0,3.0,0.0,3.0,5.0,5.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
431,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
movie_data